# 🚀 CHULA Loss Demo on Medical Pills Dataset  
*Unlock the Future with Intelligent Machines* 🤖✨

> 🌸 **CHULA**: *Custom Heuristic Uncertainty-guided Loss for Accurate Land Title Deed Segmentation*  
> 🧠 **Author**: Teerapong Panboonyuen (aka Kao Panboonyuen, ธีรพงศ์ ปานบุญยืน, เก้า ปานบุญยืน)  
> 🚩 Supported by the Second Century Fund (C2F) Postdoctoral Fellowship, Chulalongkorn University  
> 🧪 Reproducible • Plug-and-Play • Open Source for Document AI Research

---

### 📌 Note  
Due to access restrictions on Thai Land Title Deed data, this demo showcases CHULA on a public dataset (medical pills) to highlight its plug-and-play flexibility and generalization power.

---

### 🔗 Links

- 🧬 **GitHub Repository**: [kaopanboonyuen/CHULA](https://github.com/kaopanboonyuen/CHULA)  
- 🌐 **Project Page**: [kaopanboonyuen.github.io/CHULA](https://kaopanboonyuen.github.io/CHULA)  
- 📖 **Reference and Credit**: [Ultralytics Datasets Docs](https://docs.ultralytics.com/datasets/)

---

### 🧠 Citation

If you use CHULA in your research or projects, please cite:

```

@article{panboonyuen2025chula,
title={CHULA: Custom Heuristic Uncertainty-guided Loss for Accurate Land Title Deed Segmentation},
author={Panboonyuen, Teerapong},
year={2025}
}

```

---

#### 📄 License (MIT)


![](https://b2541915.smushcdn.com/2541915/wp-content/uploads/2021/01/pill-bottle.jpg?lossy=1&strip=1&webp=1)

> **Reference:**  
> Shen Clinic. (n.d.). *Why do we request our patients bring all their medication to every office visit?* Retrieved from [https://shenclinic.org/why-do-we-request-our-patients-bring-all-their-medication-every-office-visit/](https://shenclinic.org/why-do-we-request-our-patients-bring-all-their-medication-every-office-visit/)

![](https://github.com/kaopanboonyuen/panboonyuen_dataset/raw/main/public_dataset/ultralytics_dataset/medical-pills-dataset-sample-image.png)

> **Reference:**  
> Jocher, G., & Rizwan, M. (2024). *Ultralytics Datasets: Medical‑pills Detection Dataset* (Version 1.0.0) [Dataset]. AGPL‑3.0. Retrieved from [https://docs.ultralytics.com/datasets/detect/medical-pills/](https://docs.ultralytics.com/datasets/detect/medical-pills/)

# ✅ Step 1: Install YOLOv8

In [ ]:
!pip install # Please insert your code here.

# ✅ Step 2: Import libraries

In [ ]:
import os, zipfile, glob, random, cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import # Please insert your code here.
import os
import matplotlib.pyplot as plt
import random
from IPython.display import display, Image

# 📥 Step 3: Download Medical Pills Dataset

In [ ]:
!wget -O medical-pills.zip https://github.com/kaopanboonyuen/panboonyuen_dataset/raw/main/public_dataset/ultralytics_dataset/medical-pills.zip

dataset_dir = "/content/datasets/medical-pills"
os.makedirs(dataset_dir, exist_ok=True)

with zipfile.ZipFile("medical-pills.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/datasets/")

print("✅ Dataset extracted to:", dataset_dir)
print("📂 Files inside:", os.listdir(dataset_dir))

with zipfile.ZipFile("medical-pills.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/datasets/medical-pills")

print("✅ Dataset extracted to:", dataset_dir)
print("📂 Files inside:", os.listdir(dataset_dir))

# 📝 Step 4: Rewrite medical-pills.yaml for Colab path

In [ ]:
yaml_content = f"""# Medical Pill Detection Dataset (Ultralytics format)
path: {dataset_dir}

train: images/train
val: images/val
test:

# Classes (example, adjust if needed in your yaml)
names:
  0: pill

download: https://github.com/kaopanboonyuen/panboonyuen_dataset/raw/main/public_dataset/ultralytics_dataset/medical-pills.zip
"""

with open(os.path.join(dataset_dir, "medical-pills.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ Rewritten medical-pills.yaml")
!cat /content/datasets/medical-pills/medical-pills.yaml

# 👀 Step 5: Preview dataset images

In [ ]:
train_images = glob.glob(os.path.join(dataset_dir, "images/train/*.jpg"))
sample_images = random.sample(train_images, 4)

plt.figure(figsize=(12, 8))
for i, img_path in enumerate(sample_images):
    img = cv2.imread(img_path)[..., ::-1]
    plt.subplot(2, 2, i+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Train Sample {i+1}")

# Please insert your code here.

# 🚀 Step 6: Train YOLOv8 on Medical Pills Dataset

## CHULA Setup

In [ ]:
!git clone https://github.com/kaopanboonyuen/CHULA.git

In [ ]:
!pip install -e CHULA >> log_chula.logs

In [ ]:
import sys
sys.path.append('/content/CHULA')

from chula.loss import CHULALoss
from chula.utils import compute_class_weights

## CHULA Loss Definition

In [ ]:
dataset_dir = "/content/datasets/medical-pills"
num_classes = 1  # adjust
class_weights = compute_class_weights(dataset_dir, num_classes).cuda()
chula_loss = CHULALoss(class_weights=class_weights, lambda_ce=1.0, lambda_unc=0.3, lambda_heu=0.5)

## Train YOLOv8 Original

In [ ]:
# --------------------------
# Train YOLOv8 Original
# --------------------------
model_orig = YOLO("# Please insert your code here.")
results_orig = model_orig.train(
    data=f"{dataset_dir}/medical-pills.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    name="yolo_medical_pills_orig"
)

In [ ]:
results_orig = model_orig.val(save=False, plots=False)

# Option 1: dictionary of results
metrics_orig = results_orig.results_dict
print("📊 Original YOLOv8 metrics:", metrics_orig)

# Option 2: precision, recall, mAP
precision, recall, map50, map50_95 = results_orig.mean_results()
print(f"Precision={precision:.3f}, Recall={recall:.3f}, mAP50={map50:.3f}, mAP50-95={map50_95:.3f}")

## Train YOLOv8 + CHULA

In [ ]:
# --------------------------
# Train YOLOv8 + CHULA
# --------------------------
model_chula = YOLO("# Please insert your code here.")

# Patch YOLO internal loss
original_loss = model_chula.model.loss
def patched_loss(preds, targets, imgs=None):
    yolo_loss = original_loss(preds, targets, imgs)
    sigma = torch.rand_like(targets.unsqueeze(1)) * 0.1
    heuristic_masks = {0: targets==0}
    chula_term = chula_loss(preds, targets, sigma=sigma, heuristic_masks=heuristic_masks)
    return yolo_loss + 0.5 * chula_term

model_chula.model.loss = patched_loss

results_chula = model_chula.train(
    data=f"{dataset_dir}/medical-pills.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    name="yolo_medical_pills_chula"
)

In [ ]:
# --------------------------
# Evaluate Original YOLOv8
# --------------------------
results_orig = model_orig.val(save=False, plots=False)
metrics_orig = results_orig.results_dict
print("📊 Original YOLOv8 metrics:", metrics_orig)

# --------------------------
# Evaluate YOLOv8 + CHULA
# --------------------------
results_chula = model_chula.val(save=False, plots=False)
metrics_chula = results_chula.results_dict
print("📊 YOLOv8 + CHULA metrics:", metrics_chula)

# --------------------------
# Optional: Unpack mean results (precision, recall, mAP)
# --------------------------
prec_o, rec_o, map50_o, map95_o = results_orig.mean_results()
prec_c, rec_c, map50_c, map95_c = results_chula.mean_results()

print(f"Original: P={prec_o:.3f}, R={rec_o:.3f}, mAP50={map50_o:.3f}, mAP50-95={map95_o:.3f}")
print(f"CHULA:    P={prec_c:.3f}, R={rec_c:.3f}, mAP50={map50_c:.3f}, mAP50-95={map95_c:.3f}")

## Compare & Plot Metrics

In [ ]:
# --------------------------
# Compare & Plot Metrics
# --------------------------
import matplotlib.pyplot as plt

# Extract from results_dict
def extract_metrics(results_dict):
    precision = results_dict.get("metrics/precision(B)", 0.0)
    recall = results_dict.get("metrics/recall(B)", 0.0)
    map50 = results_dict.get("metrics/mAP50(B)", 0.0)
    map95 = results_dict.get("metrics/mAP50-95(B)", 0.0)
    # F1 from precision & recall
    f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
    return precision, recall, map50, map95, f1

# Get values
prec_o, rec_o, map50_o, map95_o, f1_o = extract_metrics(metrics_orig)
prec_c, rec_c, map50_c, map95_c, f1_c = extract_metrics(metrics_chula)

labels = ["Precision", "Recall", "mAP50", "mAP50-95", "F1"]
orig_vals = [prec_o, rec_o, map50_o, map95_o, f1_o]
chula_vals = [prec_c, rec_c, map50_c, map95_c, f1_c]

# Plot side-by-side bars
x = range(len(labels))
plt.figure(figsize=(10,6))
plt.bar([i-0.15 for i in x], orig_vals, width=0.3, label="YOLOv8 Original")
plt.bar([i+0.15 for i in x], chula_vals, width=0.3, label="YOLOv8 + CHULA")
plt.xticks(x, labels, fontsize=12)
plt.ylabel("Metric Score", fontsize=12)
plt.ylim(0,1)
plt.title("YOLOv8 vs YOLOv8 + CHULA Loss Comparison on Medical Pills", fontsize=14, weight="bold")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 📊 Step 7: Visualize Training Results

In [ ]:
results_path = os.path.join(model_chula.trainer.save_dir, "results.png")
if os.path.exists(results_path):
    display(Image(filename=results_path))

# 📈 Step 8: Evaluate Model Performance

In [ ]:
metrics = model_chula.val(save=True, plots=True)
print("✅ Evaluation metrics:", metrics)

eval_dir = model_chula.trainer.save_dir
for plot_name in ["confusion_matrix.png", "PR_curve.png", "F1_curve.png"]:
    plot_path = os.path.join(eval_dir, plot_name)
    if os.path.exists(plot_path):
        display(Image(filename=plot_path))

# 🔮 Step 9: Inference on a Random Validation Image

In [ ]:
val_images = glob.glob(os.path.join(dataset_dir, "# Please insert your dataset path here."))
test_img = random.choice(val_images)

# Run inference and save output
results = model_chula(test_img, save=True)
print("🔍 Prediction done on:", test_img)

# ✅ Correct way to get saved prediction image path
pred_dir = results[0].save_dir  # directory YOLO saved results
pred_img = os.path.join(pred_dir, os.path.basename(results[0].path))

display(Image(filename=pred_img))
